In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
!python3 -m pip install pandas-ods-reader

     |████████████████████████████████| 125 kB 1.7 MB/s eta 0:00:01
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49001 sha256=ec4c70a327508a83b35674e4a03c2402c974b4dffb19bd5d130a00979cc34ed1
  Stored in directory: /Users/compindia/Library/Caches/pip/wheels/d0/0e/c2/1af349f0c9e110ed08133a3d579f402f3ec1c031eac80bc084
Successfully built ezodf
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [13]:
df = pd.read_csv('dataset.csv')
df.head()

,Predicted Class,Actual Class
0,0,0
1,0,0
2,1,0
3,0,0
4,0,0


In [21]:
df.shape

(160, 2)

In [16]:
df['Predicted Class'].value_counts(),df['Actual Class'].value_counts()

(1    45
 2    39
 0    39
 3    37
 Name: Predicted Class, dtype: int64,
 0    49
 2    40
 1    36
 3    35
 Name: Actual Class, dtype: int64)

## 2.a Total Accuracy and Class wise Accuracy

In [24]:
# Total accuracy:
total_no_of_rows = df.shape[0]
true_positives = df[['Predicted Class']==df['Actual Class']]
total_accuracy = len(true_positives)/total_no_of_rows
total_accuracy

0.75625

In [74]:
# class 0 accuracy:
true_positives={}
class_accuracy={}
num_classes =len(df['Predicted Class'].value_counts())

for i in range(num_classes):
    true_positives[str(i)+str(i)] = len(df[(df['Actual Class']==i)&(df['Predicted Class'] == i)])
    class_accuracy[i] = true_positives[str(i)+str(i)]/len(df[df['Actual Class']==i])

# true_positives,
print('Class wise Accuracy: ', class_accuracy)

Class wise Accuracy:  {0: 0.673469387755102, 1: 0.8888888888888888, 2: 0.725, 3: 0.7714285714285715}


In [138]:
# Claculate False Negatives, False Positives and True Negatives

false_negatives={}
for i in range(num_classes):
    for j in range(num_classes):
        if i!=j:
            false_negatives[str(j)+str(i)] = df[(df['Actual Class']==i)&(df['Predicted Class'] == j)]


false_positives={}
for i in range(num_classes):
    for j in range(num_classes):
        if i!=j:
            false_positives[str(i)+str(j)] = df[(df['Actual Class']==j)&(df['Predicted Class'] == i)]
        
true_negatives = {}
for i in range(num_classes):
    for j in range(num_classes):
        for k in range(num_classes):
            if i!=j and i!=k:
                true_negatives[str(i)+str(k)+str(j)] = df[(df['Actual Class']==k)&(df['Predicted Class'] == j)]


## 2.b Confusion Matrix

In [139]:
# Claculate Confusion Matrix:
cf = pd.DataFrame({'actual_0':[],'actual_1':[],'actual_2':[],'actual_3':[]})
cf['actual_0']=['a','a','a','a']
cf['actual_1']=['a','a','a','a']
cf['actual_2']=['a','a','a','a']
cf['actual_3']=['a','a','a','a']

cf

,actual_0,actual_1,actual_2,actual_3
0,a,a,a,a
1,a,a,a,a
2,a,a,a,a
3,a,a,a,a


In [142]:
for i in range(num_classes):
    for j in range(num_classes):
        if i == j:
            cf['actual_'+str(i)][j] = true_positives[str(i)+str(j)]
        if i!=j:
            cf['actual_'+str(i)][j] = len(false_positives[str(j)+str(i)])
print('Confusion Matrix is: \n',cf)

Confusion Matrix is: 
   actual_0 actual_1 actual_2 actual_3
0       33        1        3        2
1        8       32        2        3
2        4        3       29        3
3        4        0        6       27


## 2.c Calculate Precision and Recall for all classes

In [143]:
true_positives

{'00': 33, '11': 32, '22': 29, '33': 27}

In [144]:
len(false_positives['01']),len(false_positives['02']),len(false_positives['03']),

(1, 3, 2)

In [145]:
# Precision = TP/TP+FP
precision={};fp={}
for i in range(num_classes):
    sum=0
    for j in range(num_classes):
        if i!=j:
            sum += len(false_positives[str(i)+str(j)])
    fp['fp_'+str(i)] = sum    
    precision[str(i)] = true_positives[str(i)+str(i)]/(true_positives[str(i)+str(i)]+sum)
print('precision',precision)

precision {'0': 0.8461538461538461, '1': 0.7111111111111111, '2': 0.7435897435897436, '3': 0.7297297297297297}


In [146]:
# Recall = TP/TP+FN

recall={};fn={}
for i in range(num_classes):
    sum=0
    for j in range(num_classes):
        if i!=j:
            sum += len(false_negatives[str(j)+str(i)])
    fn['fn_'+str(i)] = sum    
    recall[str(i)] = true_positives[str(i)+str(i)]/(true_positives[str(i)+str(i)]+sum)
print('recall',recall)

recall {'0': 0.673469387755102, '1': 0.8888888888888888, '2': 0.725, '3': 0.7714285714285715}


In [147]:
fp,fn

({'fp_0': 6, 'fp_1': 13, 'fp_2': 10, 'fp_3': 10},
 {'fn_0': 16, 'fn_1': 4, 'fn_2': 11, 'fn_3': 8})

In [148]:
# Micro-average of precision=𝑇𝑃1+𝑇𝑃2/𝑇𝑃1+𝑇𝑃2+𝐹𝑃1+𝐹𝑃2=12+5012+50+9+23=65.96

sum_tp = true_positives['00']+true_positives['11']+true_positives['22']+true_positives['33']
sum_fp = fp['fp_0']+fp['fp_1']+fp['fp_2']+fp['fp_3']
sum_fn = fn['fn_0']+fn['fn_1']+fn['fn_2']+fn['fn_3']

micro_average_precision = sum_tp/(sum_tp+sum_fp)
print('micro_average_precision : ',micro_average_precision)

micro_average_recall = sum_tp/(sum_tp+sum_fn)
print('micro_average_recall : ',micro_average_recall)


macro_average_precision = (precision['0']+precision['1']+precision['2']+precision['3'])/4
macro_average_recall = (recall['0']+recall['1']+recall['2']+recall['3'])/4

print('macro_average_precision : ',macro_average_precision)
print('macro_average_recall : ',macro_average_recall)

micro_average_precision :  0.75625
micro_average_recall :  0.75625
macro_average_precision :  0.7576461076461076
macro_average_recall :  0.7646967120181406


## 2.d F1-Score

In [149]:
# F!-Score:   2/((1/precision)+(1/recall))

f1_score={}
for i in range(num_classes):
    f1_score[i] = 2/((1/precision[str(i)])+(1/recall[str(i)]))
f1_score

{0: 0.7499999999999999,
 1: 0.7901234567901234,
 2: 0.7341772151898733,
 3: 0.7499999999999999}

## 2.e Type-1 and Type-2 Errors

In [150]:
# Type-1: Fp/Fp+TN , Type-2: FN/FN+TP

true_negatives.keys()

dict_keys(['011', '021', '031', '012', '022', '032', '013', '023', '033', '100', '120', '130', '102', '122', '132', '103', '123', '133', '200', '210', '230', '201', '211', '231', '203', '213', '233', '300', '310', '320', '301', '311', '321', '302', '312', '322'])

In [151]:
tn_keys = {'0':['011', '021', '031', '012', '022', '032', '013', '023', '033'],
           '1':['100', '120', '130', '102', '122', '132', '103', '123', '133'],
           '2':['200', '210', '230', '201', '211', '231', '203', '213', '233'],
           '3':['300', '310', '320', '301', '311', '321', '302', '312', '322']}
tn_sum={}
for i,j in tn_keys.items():
    sum=0
    for k in j:
        sum+=len(true_negatives[k])
    tn_sum[i]=sum
tn_sum

{'0': 105, '1': 111, '2': 110, '3': 115}

In [152]:
fn,fp

({'fn_0': 16, 'fn_1': 4, 'fn_2': 11, 'fn_3': 8},
 {'fp_0': 6, 'fp_1': 13, 'fp_2': 10, 'fp_3': 10})

In [154]:
type_1={}
type_2={}
for i in range(num_classes):
    type_1[i] = fp['fp_'+str(i)]/(fp['fp_'+str(i)]+tn_sum[str(i)])
for i in range(num_classes):
    type_2[i] = fn['fn_'+str(i)]/(fn['fn_'+str(i)]+true_positives[str(i)+str(i)])
print('type-1 error: \n',type_1)
print('\ntype-2 error: \n',type_2)

type-1 error: 
 {0: 0.05405405405405406, 1: 0.10483870967741936, 2: 0.08333333333333333, 3: 0.08}

type-2 error: 
 {0: 0.32653061224489793, 1: 0.1111111111111111, 2: 0.275, 3: 0.22857142857142856}


In [156]:
y_test = df['Actual Class']
y_pred = df['Predicted Class']


#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_pred,y_test)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))


print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Class 0','Class 1', 'Class 2', 'Class 3']))

Confusion Matrix

[[33  1  3  2]
 [ 8 32  2  3]
 [ 4  3 29  3]
 [ 4  0  6 27]]

Accuracy: 0.76

Macro Precision: 0.76
Macro Recall: 0.76
Macro F1-score: 0.76

Weighted Precision: 0.76
Weighted Recall: 0.76
Weighted F1-score: 0.76

Classification Report

              precision    recall  f1-score   support

     Class 0       0.85      0.67      0.75        49
     Class 1       0.71      0.89      0.79        36
     Class 2       0.74      0.72      0.73        40
     Class 3       0.73      0.77      0.75        35

    accuracy                           0.76       160
   macro avg       0.76      0.76      0.76       160
weighted avg       0.76      0.76      0.76       160



In [157]:
from sklearn.metrics import log_loss
accuracy = log_loss(y_test, y_pred)

ValueError: y_true and y_pred contain different number of classes 4, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3]